<div style="text-align: center">
<h1 ><img src="https://media.giphy.com/media/LMt9638dO8dftAjtco/giphy.gif" style = "height:38px"/> Carga de datos <img src="https://media.giphy.com/media/TEdRZnV7l3S067fiGR/giphy.gif" style = "height:38px"/></h1 >
</div>


Las tarjetas de FiguRace se generan a partir de un conjunto de datos que deberán ser preparados previamente. Como parte del proyecto se deberá desarrollar un cuaderno de Jupyter Notebook que extraiga los datos necesarios para el juego de alguno de los datasets provistos y muestre en forma detallada el proceso realizado con los mismos . En el Anexo II se detallan los datasets que deberán utilizar para la aplicación y las modificaciones necesarias para luego guardarlo en un nuevo archivo en formato CSV. El juego deberá cargar esos archivos CSV para funcionar


<h3 align="center"><img src="https://media.giphy.com/media/F3IsWfsR1JMyNmL44Z/giphy.gif" style = "height:38px" /> Spotify 2010 - 2019 Top 100 <img  src="https://media.giphy.com/media/F3IsWfsR1JMyNmL44Z/giphy.gif" style = "height:38px" /></h3>


Deberá adaptar los datos de la siguiente forma:

- Poner en `title case` los géneros musicales excepto las siglas EDM, DFW, UK, R&B y LGBTQ+ que deben ir en mayúsculas. Por ejemplo `dfw rap` debe ser transformado a `DFW Rap`.
- Considerar también la excepción `k-pop` que debe ser transformada a `K-Pop`.
- Se utilizarán como datos de las tarjetas `Top Genre`, `Year Released`, `BPM`, `Top Year` y `Artist Type`. Como dato a adivinar se utilizará `Artist`. Descartar el resto de las columnas.
- El archivo resultante deberá tener las siguientes columnas (en este orden específico): `Top Genre`, `Artist Type`, `Year Released`, `Top Year`, `BPM` y `Artist`


Declaramos las constantes
--------------------------

- `UPPER_GENDERS` : Contiene la lista de palabras que tenemos que convertir completamente a mayuscula
- `ORDEN` : El orden en el que tienen que quedar las columnas
- `NAMES_REMOVE` : Las columnas que tengo que dejar en el archivo

In [50]:
# Constantes y variables globales
import os
import csv
UPPER_GENDERS = ["EDM", "DFW", "UK", "R&B", "LGBTQ+"]
ORDEN = ["Top Genre", "Artist Type", "Year Released", "Top Year", "BPM" ,"Artist"]
NAMES_REMOVE = ["Artist", "Top Genre", "Year Released","BPM", "Top Year", "Artist Type"]
path = os.path.dirname(os.path.realpath("."))
path

'C:\\Users\\fabian\\Desktop\\PysimpleGuy\\grupo27'

### Procesando los datos

Separamos el encabezado de el resto del archivo ya que mas adelante vamos a utilizarlo/modificarlo


In [51]:
def data_process(path:str):
    header:list[str] = []
    datos:list[list[str]] = []
    path_file = os.path.join(path,"dataset_section", "base_datasets", 'Spotify_2010-2019_Top_100.csv')
    with open(path_file,"r", encoding="utf-8") as File:  
        csv_reader = csv.reader(File, delimiter=',')
        header = next(csv_reader) # El encabezado para comparar las columnas borradas
        datos = list(csv_reader)
    return header,datos

header,datos = data_process(path)
header

['\ufefftitle',
 'artist',
 'top genre',
 'year released',
 'added',
 'bpm',
 'nrgy',
 'dnce',
 'dB',
 'live',
 'val',
 'dur',
 'acous',
 'spch',
 'pop',
 'top year',
 'artist type']

Tenemos una constante que contiene el orden en el que tenemos que dejar las columnas `ORDEN`.

- Pasamos esta lista a minuscula para comparar y la recorremos
- Cargamos la lista con los indices


In [52]:
def create_order(orden_list:list[int]):
    # Columnas para invertir datos
    """Agregamos a una lista, los indices manteniendo un orden.
    """
    names_compare = list(map(lambda x: x.lower(), ORDEN))
    for name in names_compare:
        for pos,dato2 in enumerate(header):
            if (name == dato2):
                orden_list.append(pos)

orden_list:list[int] = []  
create_order(orden_list)
orden_list

[2, 16, 3, 15, 5, 1]

### Filtrado de Columnas

- Separamos las columnas que queremos mantener de las que vamos a eliminar
- Generamos sus respectivas listas, con los indices para reutilizar el codigo en un futuro


In [53]:
def separate_cols(cols_remove:list[int],cols_saves:list[int]):
    names_compare = list(map(lambda x: x.lower(), NAMES_REMOVE)) # Creamos una lista para comparar
    for column in header:
        if not(column.lower() in names_compare): 
            index_dato = header.index(column)
            cols_remove.append(index_dato)
        else:                                 # Para ordenar los datos entre listas
            index_dato2 = header.index(column)
            cols_saves.append(index_dato2)


cols_remove:list[int] = []  # Columnas que voy a eliminar [0, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14] 
cols_saves:list[int] = []   # Columnas que voy a guardar  [1, 2, 3, 5, 15, 16] 
separate_cols(cols_remove,cols_saves)
cols_remove,cols_saves

([0, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14], [1, 2, 3, 5, 15, 16])

### Ordenamos Columnas

- Recorremos todo el archivo
  - Recorremos una fila
    - Ordenamos las columnas dependiendo de cual sea su indice


In [54]:
def cols_sorted(datos:list[list[str]],orden_list:list[int]):
    for row in datos:
        # Recorro la lista de columnas que tengo que intercambiar 
        # Despues intercambio los datos entre columnas
        for col_index in cols_saves:                        # [1, 2, 3, 5, 15, 16]
            for indice,dato in enumerate(orden_list):     
                if (col_index == dato):                     # [2, 16, 3, 15, 5, 1]
                    aux = row[dato]
                    row[dato]= row[indice]                  # En lugar de col_index,es la posicion de la tabla
                    row[indice] = aux
    return datos

### Mayusculas-Minusculas

- Con la frase que tenemos, generamos una lista de palabras para poder trabajar en cada una de ellas por separado
- Recorremos la lista
  - Si se encuentra en la lista, la convertimos toda a mayuscula
  - Sino usamos `title()` para pasar la primera a mayuscula


In [55]:
def upper_words(word:str):
    genders = word.split()
    for index, gender in enumerate(genders):
        if gender.upper() in UPPER_GENDERS:
            genders[index]= gender.upper()
        else:
            genders[index]= gender.title()
    word = " ".join(genders)
    return word

### Funcion para eliminar las columnas


In [56]:
def cols_remove_function(row:list[str],cols_remove:list[int]):
    # Elimino las columnas innecesarias
    for col_index in cols_remove:
        del (row[col_index])
    return row

### Programa


In [57]:
cols_remove = sorted(cols_remove, reverse=True) # En reverso para sacar primero las columnas desde el final
path_newfile = os.path.join(path, "dataset_section", "processed_datasets", 'Spotify.processed.csv')

with open(path_newfile, "w",encoding="utf-8",newline='') as NewFile:
    csv_writer = csv.writer(NewFile)
    datos.insert(0,header)             # Vuelvo a insertar el encabezado para eliminar las columnas
    datos = cols_sorted(datos,orden_list)
    for row in datos:
        row[2] = upper_words(row[2])
        row = cols_remove_function(row,cols_remove)
        csv_writer.writerow(row)
header     

['artist type', 'Year Released', 'top year', 'added', '\ufefftitle', 'bpm']

<h3 align="center"><img src="https://media.giphy.com/media/QWrCFpI965negKDo0n/giphy.gif" style = "height:38px" /> Lagos Argentina - Hoja 1 <img  src="https://media.giphy.com/media/QWrCFpI965negKDo0n/giphy.gif" style = "height:38px" /></h3>


Deberá adaptar los datos de la siguiente forma:

- Transformar las coordenadas en la columna `Coordenadas` a grados decimales.
- Se utilizarán como datos de las tarjetas: `Ubicación`, `Superficie (km²)`, `Profundidad máxima (m)`, `Profundidad media (m)`, `Coordenadas`. Como dato a adivinar se utilizará `Nombre`. Descartar el resto de las columnas.
- El archivo resultante deberá tener las siguientes columnas (en este orden específico): `Ubicación`, `Superficie (km²)`, `Profundidad máxima (m)`, `Profundidad media (m)`, `Coordenadas` y `Nombre`.


<h3 align="center"><img src="https://media.giphy.com/media/W55QKlBohHZS9RGOWJ/giphy.gif" style = "height:38px" /> FIFA-21 Complete <img  src="https://media.giphy.com/media/W55QKlBohHZS9RGOWJ/giphy.gif" style = "height:38px" /></h3>


Deberá adaptar los datos de la siguiente forma:

- Reemplazar `Potential` por la siguiente escala conceptual:
  - Regular: Menos de 60
  - Bueno: Entre 60 y 79 (inclusive)
  - Muy bueno: Entre 80 y 89 (inclusive)
  - Sobresaliente: Desde 90 en adelante.
- Reemplazar el valor de `Position` por las posiciones en español. Por ejemplo `LB|CB` debe ser reemplazado por `Defensor izquierdo|Defensor central`
- Se utilizarán como datos de las tarjetas: `Age`, `Nationality`, `Position`, `Team` y `Potential`. Como dato a adivinar se utilizará `Name`. Descartar el resto de las columnas.
- El archivo resultante deberá tener las siguientes columnas (en este orden específico): `Team`, `Nationality`, `Position`, `Age`, `Potential` y `Name`.
